# **Model Training Process**

**1) Using Ultralytics as shortcut of whole pytorch modelling procedure.**

In [ ]:
# Install YOLOv8 (Ultralytics)
!pip install ultralytics

# Check if installed
import ultralytics
ultralytics.checks()


Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 41.9/112.6 GB disk)


**2) Mouting Google drive as data source
(Data downloaded from kaggle ; Link : https://www.kaggle.com/datasets/ziya07/multi-class-fabric-defect-detection-dataset)**

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


**3) Creating a file called data.yaml in Colab notebook**

In [ ]:
yaml_content = """
train: /content/drive/MyDrive/Colab_Notebooks/Data-set/images/train
val: /content/drive/MyDrive/Colab_Notebooks/Data-set/images/val
test: /content/drive/MyDrive/Colab_Notebooks/Data-set/images/test

nc: 6
names: [ 'Broken stitch', 'Hole', 'Lines', 'Needle mark', 'Pinched fabric', 'Stain' ]
"""

with open("fabric_data.yaml", "w") as f:
    f.write(yaml_content)


Remapping class numbers as i have deleted two classes (Horizontals & Vertical)

In [ ]:
import os

# Class index remapping
class_map = {
    0: 0,  # Broken stitch
    1: 1,  # Hole
    3: 2,  # Lines
    4: 3,  # Needle mark
    5: 4,  # Pinched fabric
    6: 5,  # Stain
}

def remap_classes(label_folder):
    for fname in os.listdir(label_folder):
        if not fname.endswith(".txt"):
            continue
        path = os.path.join(label_folder, fname)
        with open(path, 'r') as f:
            lines = f.readlines()

        updated = []
        for line in lines:
            if not line.strip(): continue
            parts = line.strip().split()
            cls = int(parts[0])
            if cls in class_map:
                parts[0] = str(class_map[cls])
                updated.append(" ".join(parts))
            else:
                # Skip removed classes
                continue

        with open(path, 'w') as f:
            f.write("\n".join(updated) + "\n")

# Run on all 3 sets
remap_classes('/content/drive/MyDrive/Colab_Notebooks/Data-set/labels/train')
remap_classes('/content/drive/MyDrive/Colab_Notebooks/Data-set/labels/val')
remap_classes('/content/drive/MyDrive/Colab_Notebooks/Data-set/labels/test')


**4) Data Preprocessing
[Annotations (labels of images created using Cvat.ai) and organising raw data from kaggle into test,train and Validation already done and uploaded in drive. Here only processes line identifying and cleaning is done]**

In [ ]:
import os

label_dir = "/content/drive/MyDrive/Colab_Notebooks/Data-set/labels/train"
bad_labels = []

for fname in os.listdir(label_dir):
    if fname.endswith(".txt"):
        path = os.path.join(label_dir, fname)
        with open(path, 'r') as f:
            lines = f.readlines()
            for line in lines:
                parts = line.strip().split()
                if len(parts) != 5:
                    bad_labels.append(fname)
                    break
                if not all(p.replace('.', '', 1).isdigit() or p.isdigit() for p in parts):
                    bad_labels.append(fname)
                    break
                if int(parts[0]) >= 8:
                    bad_labels.append(fname)
                    break

print(f"Corrupt or invalid label files ({len(bad_labels)}):\n", bad_labels)


Corrupt or invalid label files (89):
 ['Needle mark_A_08_002.txt', 'Needle mark_A_08_087.txt', 'Needle mark_A_08_021.txt', 'Needle mark_A_08_103.txt', 'Needle mark_A_08_107.txt', 'Needle mark_A_08_003.txt', 'Needle mark_A_08_013.txt', 'Needle mark_A_08_077.txt', 'Needle mark_A_08_094.txt', 'Needle mark_A_08_008.txt', 'Needle mark_A_08_053.txt', 'Pinched fabric_A_03_025.txt', 'Needle mark_A_08_014.txt', 'Needle mark_A_08_106.txt', 'Pinched fabric_A_03_020.txt', 'Needle mark_A_08_048.txt', 'Needle mark_A_08_108.txt', 'Needle mark_A_08_075.txt', 'Needle mark_A_08_044.txt', 'Needle mark_A_08_025.txt', 'Needle mark_A_08_031.txt', 'Needle mark_A_08_093.txt', 'Needle mark_A_08_060.txt', 'Pinched fabric_A_03_016.txt', 'Needle mark_A_08_102.txt', 'Pinched fabric_A_03_021.txt', 'Needle mark_A_08_049.txt', 'Needle mark_A_08_019.txt', 'Needle mark_A_08_020.txt', 'Needle mark_A_08_066.txt', 'Needle mark_A_08_074.txt', 'Pinched fabric_A_03_024.txt', 'Needle mark_A_08_011.txt', 'Needle mark_A_08_041.

Missing annotations does't affect model infact helps in training so it is not erased.

In [ ]:
image_dir = "/content/drive/MyDrive/Colab_Notebooks/Data-set/images/train"
label_dir = "/content/drive/MyDrive/Colab_Notebooks/Data-set/labels/train"

import os

image_files = [f[:-4] for f in os.listdir(image_dir) if f.endswith(('.jpg', '.png', '.jpeg'))]
label_files = [f[:-4] for f in os.listdir(label_dir) if f.endswith('.txt')]

missing_labels = [img for img in image_files if img not in label_files]
empty_labels = [f for f in label_files if os.path.getsize(os.path.join(label_dir, f + ".txt")) == 0]

print(f"Images with missing labels: {len(missing_labels)}")
print("Sample missing:", missing_labels[:5])
print(f"Empty label files (no annotations): {len(empty_labels)}")
print("Sample empty:", empty_labels[:5])


Images with missing labels: 0
Sample missing: []
Empty label files (no annotations): 0
Sample empty: []


**I have not added defect free images so i am adding that along with creating there labels**

In [ ]:
import os
import random
import shutil

# ✅ SET YOUR PATHS (Update if needed)
source_images_path = "/content/drive/MyDrive/Colab_Notebooks/Data-set/images/defect free"
target_images_path = "/content/drive/MyDrive/Colab_Notebooks/Data-set/images/train"
target_labels_path = "/content/drive/MyDrive/Colab_Notebooks/Data-set/labels/train"

# ✅ Check if source folder exists
if not os.path.exists(source_images_path):
    raise FileNotFoundError(f"Source folder not found: {source_images_path}")

# ✅ Make sure target folders exist
os.makedirs(target_images_path, exist_ok=True)
os.makedirs(target_labels_path, exist_ok=True)

# ✅ Step 1: Collect image filenames
all_images = [f for f in os.listdir(source_images_path) if f.lower().endswith((".jpg", ".jpeg", ".png"))]

# ✅ Step 2: Randomly select 500
selected_images = random.sample(all_images, 500)

# ✅ Step 3: Copy images + create empty label files
for img_file in selected_images:
    src_img = os.path.join(source_images_path, img_file)
    dst_img = os.path.join(target_images_path, img_file)
    shutil.copy(src_img, dst_img)

    label_name = os.path.splitext(img_file)[0] + ".txt"
    dst_label = os.path.join(target_labels_path, label_name)
    with open(dst_label, "w") as f:
        pass  # creates an empty label file

print(f"✅ Successfully copied 500 defect-free images and created empty labels.")


✅ Successfully copied 500 defect-free images and created empty labels.


Deleting Empty labels of train

In [ ]:
import os

# Set paths
labels_dir = '/content/drive/MyDrive/Colab_Notebooks/Data-set/labels/train'  # or val/test
images_dir = '/content/drive/MyDrive/Colab_Notebooks/Data-set/images/train'  # or val/test

# Get list of image filenames without extensions
image_basenames = {os.path.splitext(f)[0] for f in os.listdir(images_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))}

# Loop through label files
for label_file in os.listdir(labels_dir):
    label_basename = os.path.splitext(label_file)[0]
    if label_basename not in image_basenames:
        # Orphan label found — delete it
        os.remove(os.path.join(labels_dir, label_file))
        print(f"Deleted orphan label: {label_file}")

Deleted orphan label: stain_114.txt
Deleted orphan label: stain_108.txt
Deleted orphan label: stain_109.txt
Deleted orphan label: stain_102.txt
Deleted orphan label: stain_105.txt
Deleted orphan label: stain_101.txt
Deleted orphan label: stain_111.txt
Deleted orphan label: stain_107.txt
Deleted orphan label: stain_10.txt
Deleted orphan label: stain_139.txt
Deleted orphan label: stain_131.txt
Deleted orphan label: stain_121.txt
Deleted orphan label: stain_124.txt
Deleted orphan label: stain_126.txt
Deleted orphan label: stain_123.txt
Deleted orphan label: stain_135.txt
Deleted orphan label: stain_132.txt
Deleted orphan label: stain_133.txt
Deleted orphan label: stain_130.txt
Deleted orphan label: stain_138.txt
Deleted orphan label: stain_140.txt
Deleted orphan label: stain_142.txt
Deleted orphan label: stain_227.txt
Deleted orphan label: stain_166.txt
Deleted orphan label: stain_171.txt
Deleted orphan label: stain_210.txt
Deleted orphan label: stain_182.txt
Deleted orphan label: stain_1

In [ ]:
import os

# Set paths
labels_dir = '/content/drive/MyDrive/Colab_Notebooks/Data-set/labels/test'  # or val/test
images_dir = '/content/drive/MyDrive/Colab_Notebooks/Data-set/images/test'  # or val/test

# Get list of image filenames without extensions
image_basenames = {os.path.splitext(f)[0] for f in os.listdir(images_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))}

# Loop through label files
for label_file in os.listdir(labels_dir):
    label_basename = os.path.splitext(label_file)[0]
    if label_basename not in image_basenames:
        # Orphan label found — delete it
        os.remove(os.path.join(labels_dir, label_file))
        print(f"Deleted orphan label: {label_file}")


Deleted orphan label: stain_353.txt
Deleted orphan label: stain_40.txt
Deleted orphan label: stain_240.txt
Deleted orphan label: stain_396.txt
Deleted orphan label: stain_134.txt
Deleted orphan label: stain_144.txt
Deleted orphan label: stain_55.txt
Deleted orphan label: stain_245.txt
Deleted orphan label: stain_207.txt
Deleted orphan label: stain_141.txt
Deleted orphan label: stain_323.txt
Deleted orphan label: stain_54.txt
Deleted orphan label: stain_122.txt
Deleted orphan label: stain_41.txt
Deleted orphan label: stain_377.txt
Deleted orphan label: stain_394.txt
Deleted orphan label: stain_250.txt
Deleted orphan label: stain_128.txt
Deleted orphan label: stain_146.txt
Deleted orphan label: stain_251.txt
Deleted orphan label: stain_371.txt


In [ ]:
import os

# Set paths
labels_dir = '/content/drive/MyDrive/Colab_Notebooks/Data-set/labels/val'  # or val/test
images_dir = '/content/drive/MyDrive/Colab_Notebooks/Data-set/images/val'  # or val/test

# Get list of image filenames without extensions
image_basenames = {os.path.splitext(f)[0] for f in os.listdir(images_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))}

# Loop through label files
for label_file in os.listdir(labels_dir):
    label_basename = os.path.splitext(label_file)[0]
    if label_basename not in image_basenames:
        # Orphan label found — delete it
        os.remove(os.path.join(labels_dir, label_file))
        print(f"Deleted orphan label: {label_file}")

Deleted orphan label: stain_110.txt
Deleted orphan label: stain_125.txt
Deleted orphan label: stain_136.txt
Deleted orphan label: stain_100.txt
Deleted orphan label: stain_129.txt
Deleted orphan label: stain_113.txt
Deleted orphan label: stain_127.txt
Deleted orphan label: stain_60.txt
Deleted orphan label: stain_43.txt
Deleted orphan label: stain_153.txt
Deleted orphan label: stain_348.txt
Deleted orphan label: stain_262.txt
Deleted orphan label: stain_30.txt
Deleted orphan label: stain_307.txt
Deleted orphan label: stain_299.txt
Deleted orphan label: stain_383.txt
Deleted orphan label: stain_356.txt
Deleted orphan label: stain_45.txt
Deleted orphan label: stain_76.txt
Deleted orphan label: stain_239.txt
Deleted orphan label: stain_308.txt
Deleted orphan label: stain_357.txt
Deleted orphan label: stain_366.txt
Deleted orphan label: stain_247.txt
Deleted orphan label: stain_23.txt
Deleted orphan label: stain_325.txt
Deleted orphan label: stain_50.txt
Deleted orphan label: stain_56.txt


**5) Training Model**

In [ ]:
from ultralytics import YOLO
model = YOLO("yolov8s.pt")  # or 'yolov8n.pt' for faster, 'yolov8m.pt' for better accuracy

model.train(
    data="fabric_data.yaml",
    epochs=75,
    imgsz=640,                # better quality images
    batch=8,                  # adjust based on GPU RAM (use 16 if high memory)
    patience=20,              # early stopping
    cache=True,               # speeds up training
    workers=2,                # for Google Colab
    device=0,                 # auto-selects GPU
    amp=True,                 # mixed precision training
    lr0=0.01,                 # learning rate
    weight_decay=0.0005,
    hsv_h=0.015,              # color augmentations
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=0.2,
    translate=0.1,
    scale=0.5,
    shear=0.1,
    perspective=0.0001,
    flipud=0.5,
    fliplr=0.5,
    mosaic=1.0,
    mixup=0.2,
    copy_paste=0.2,
    project="fabric-defect-exp",
    name="yolov8s-fabric",
    pretrained=True,
    close_mosaic=10,         # disables mosaic in last epochs
    val=True                 # run validation at end
)


Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.2, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=fabric_data.yaml, degrees=0.2, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=75, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.5, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.2, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8s-fabric2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=20, perspective=0.0001, plots=True, pose=12.0, pretrained=True

100%|██████████| 755k/755k [00:00<00:00, 20.2MB/s]

Overriding model.yaml nc=80 with nc=6

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

 22        [15, 18, 21]  1   2118370  ultralytics.nn.modules.head.Detect           [6, [128, 256, 512]]          
Model summary: 129 layers, 11,137,922 parameters, 11,137,906 gradients, 28.7 GFLOPs

Transferred 349/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


100%|██████████| 5.35M/5.35M [00:00<00:00, 88.3MB/s]


...

      75/75      6.69G      1.565       1.29      1.622          4        640: 100%|██████████| 131/131 [00:23<00:00,  5.62it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:01<00:00,  8.57it/s]


                   all        150        235       0.46      0.428      0.411       0.19

75 epochs completed in 0.557 hours.
Optimizer stripped from fabric-defect-exp/yolov8s-fabric2/weights/last.pt, 22.5MB
Optimizer stripped from fabric-defect-exp/yolov8s-fabric2/weights/best.pt, 22.5MB

Validating fabric-defect-exp/yolov8s-fabric2/weights/best.pt...
Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,127,906 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:01<00:00,  5.43it/s]


                   all        150        235      0.508      0.453      0.443      0.204
         Broken stitch         22         23      0.761      0.832      0.846      0.425
                  Hole         29         36      0.348      0.361      0.275      0.122
                 Lines         32         46      0.438      0.322      0.202     0.0767
           Needle mark          2          2          0          0          0          0
        Pinched fabric         21         28       0.76      0.643      0.737      0.318
                 Stain         29        100       0.74       0.56        0.6      0.284
Speed: 0.5ms preprocess, 4.2ms inference, 0.0ms loss, 2.7ms postprocess per image
Results saved to fabric-defect-exp/yolov8s-fabric2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7cafab606610>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
     